<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Module Installation </span>

In [1]:
# !pip install bs4
# !pip install requests
# !pip install pandas


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Package Imports </span>

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Crawl and Scrape tracks from specific artists </span>

In [ ]:
%run py_utils/scraping_utils.ipynb
%run py_utils/generic_utils.ipynb
%run py_utils/sqlite_utils.ipynb

def tracks_crawler(start_urls):
    track_list = []
    artist_list = []
    album_list = []
    url_list = []
    
    db_flag = 1
    db_table = 'soundtrack_table'
    db_path = '../database/db_soundtracks.db'
    connection = create_connection(db_path)
    cursor = create_cursor(connection)    
    create_table_db(['tracks', 'text'], ['artists', 'text'], ['albums', 'text'], ['urls', 'text'], cursor_arg=cursor, 
                 table_name=db_table)
    
    a_key = 'a'
    div_key = 'div'
    artist_key = 'title'

    href_attr = {'class': 'listalbum-item'}
    album_attr = {'class': 'album'}
    tracks_table = {'id': 'listAlbum'}
    album_class = ['album']
    track_class = ['listalbum-item']
    
    for url in start_urls:
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'lxml') 
        
        try:
            find_table = bs4_scrape(soup, 'find', div_key, tracks_table)
            find_lists = bs4_scrape(find_table, 'findAll', div_key)    

        except Exception as exc:
            find_lists = []
            print('Exception Handled: ', exc)
            temp = bs4_scrape(soup, 'findAll', div_key, href_attr)
            
            for i in temp:
                find_lists.append(i)
        
        artist_name = bs4_scrape(soup, 'find', artist_key).text        
        artist_name = artist_name.replace("'", "")
        tracks, artists, albums, urls = get_track_info(find_lists, album_class, track_class, artist_name, a_key)

        for index in range(len(tracks)):
            try:
                insert_values_db(tracks[index], artists[index], albums[index], urls[index], cursor_arg=cursor, 
                                 table_name=db_table)
                connection.commit()
                
            except Exception as exc:
                db_flag = 0
                print(exc)
            
            track_list.append(tracks[index])
            artist_list.append(artists[index])
            album_list.append(albums[index])
            url_list.append(urls[index])

    if db_flag == 1:
        print("> Database Storage Successful.")      
        
    json_file_path = '../datasets/discography_data.json'
    excel_file_path = '../datasets/discography_data.xlsx'
    csv_file_path = '../datasets/discography_data.csv'

    key_list = ['Tracks', 'Artists', 'Albums', 'URLs']
    values_list = [track_list, artist_list, album_list, url_list]
    track_info_dict = create_dict(key_list, values_list)
    df = pd.DataFrame(track_info_dict, columns=key_list)

    save_json(json_file_path, track_info_dict)
    save_df(df, excel_file_path, 'xlsx')
    save_df(df, csv_file_path, 'csv')
    connection.close()
    